In [ ]:

import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role
from pandas import read_csv
from urllib.parse import urlparse
import os
import boto3
    
###Turn off warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
if type(tensorflow.contrib) != type(tensorflow): tensorflow.contrib._warning = None

In [ ]:
s3Bucket = "XYZ-S3Bucket-XYZ"
dataset = "XYZ-IotAnalyticsDataset-XYZ"

iotanalytics_client = boto3.client('iotanalytics')
dataset_url = iotanalytics_client.get_dataset_content(datasetName = dataset)['entries'][0]['dataURI']

dataset = read_csv(dataset_url, header=0, index_col='date')
if dataset.empty:
    raise Exception('No data found')
dataset.sort_index(inplace=True)
#dataset = dataset[['pollution', 'dew', 'temp', 'press', 'wnd_dir', 'wnd_spd', 'snow', 'rain']]
dataset.drop_duplicates(inplace=True)
dataset.to_csv('pollution.csv')

In [ ]:
###Upload this queried data to s3
sagemaker_session = sagemaker.Session()
uploaded_data_path = sagemaker_session.upload_data(path='pollution.csv', bucket=s3Bucket, key_prefix='data')

In [ ]:
train_instance_type = "ml.m5.xlarge"
    
tf_estimator = TensorFlow(entry_point='trainLSTM.py', role=get_execution_role(),
                          train_instance_count=1, train_instance_type=train_instance_type,
                          framework_version='1.12', py_version='py3', script_mode=True,
                          output_path = 's3://' + s3Bucket, base_job_name = "pollution-forecasting-lstm",
                          hyperparameters={'batch_size': 72,
                                           'epochs': 50,
                                           'n_train_hours': n_train_hours,
                                           'n_validation_hours': n_validation_hours})

tf_estimator.fit(uploaded_data_path)